# Data Preparation
Here we merge the different CSV data Files


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
import pandas as pd
import os


## Join Data

In [2]:

#merge Traffic Accident Data from 2016 to 2022
folder_path = "C:\Projekte\TDS\TDS2324-TrafficAccidents\Data\TrafficAccidentData"
file_list = os.listdir(folder_path)
merged_data = pd.DataFrame()

# Iterate over each file in the folder and add to merged_data
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path,sep=';')
    merged_data = pd.concat([merged_data, data], ignore_index=False)

print(merged_data.columns)

C:\Users\miche\AppData\Local\Temp\ipykernel_29824\1934375299.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path,sep=';')
C:\Users\miche\AppData\Local\Temp\ipykernel_29824\1934375299.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path,sep=';')


Index(['FID', 'OBJECTID', 'ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR',
       'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1',
       'ULICHTVERH', 'STRZUSTAND', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad',
       'IstGkfz', 'IstSonstige', 'LINREFX', 'LINREFY', 'XGCSWGS84',
       'YGCSWGS84', 'UIDENTSTLA', 'OBJECTID_1', 'UIDENTSTLAE'],
      dtype='object')


## Clean Data
Here we will clean data and check for na

In [3]:
merged_data.drop(['FID', 'OBJECTID', 'OBJECTID_1', 'UIDENTSTLA', 'UIDENTSTLAE'], axis=1, inplace=True)
print(merged_data.columns)



Index(['ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR', 'UMONAT', 'USTUNDE',
       'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'STRZUSTAND',
       'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige',
       'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84'],
      dtype='object')


In [4]:
#save the merged data to a csv file in folder TrafficAccidentData
merged_data.to_csv("C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\all_16_22.csv")

## Introducing the Dataset

TODO: 
- Binäre Features zu Boolean ändern?
- Kategorien (STRZUSTAND, ULICHTVERH, etc..) in Kategorische Daten  ändern?
- Koordinaten von Object in Double ändern?


In [5]:
merged_data.head(10)

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,1,0,53,120,2016,1,9,5,2,8,...,0,1,0,0,0.0,0,"606982,393999999970000","5954659,924999999800000","10,621659329000000","53,729614888000000"
1,1,0,57,10,2016,1,17,3,3,1,...,0,1,0,0,0.0,0,"574882,533000000050000","6011440,850999999800000","10,149175783000000","54,245452583999999"
2,1,0,62,8,2016,1,0,5,3,9,...,0,1,0,0,0.0,0,"599934,687500000000000","5964608,596199999600000","10,518094344000000","53,820402504999997"
3,1,0,3,0,2016,1,15,5,3,5,...,1,0,0,0,0.0,1,"610709,348699999970000","5968284,242300000000000","10,683020702000000","53,851243101000001"
4,1,0,55,28,2016,1,14,1,3,8,...,0,1,0,0,0.0,0,"605690,790400000060000","6009152,214700000400000","10,620986018000000","54,219458582000001"
5,1,0,61,44,2016,1,17,6,3,9,...,0,1,0,0,0.0,0,"542547,328499999950000","5963921,747000000400000","9,646338030000001","53,822093844999998"
6,1,0,60,54,2016,1,17,3,3,6,...,1,0,1,0,0.0,0,"558412,407499999970000","5969889,909599999900000","9,888452573000000","53,874189672000000"
7,1,0,55,29,2016,1,11,1,2,2,...,0,1,0,0,0.0,0,"623911,704400000050000","6008211,261900000300000","10,899895740000000","54,206924608999998"
8,1,0,60,85,2016,1,16,2,2,4,...,0,1,0,0,0.0,0,"579664,308100000020000","5965471,277499999900000","10,210478533000000","53,831655902000001"
9,1,0,54,126,2016,1,10,4,2,5,...,0,1,0,0,0.0,1,"494320,527400000020000","6066809,662200000100000","8,911765602999999","54,748506792999997"


In [6]:
merged_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1554834 entries, 0 to 256491
Data columns (total 23 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   ULAND        1554834 non-null  int64  
 1   UREGBEZ      1554834 non-null  int64  
 2   UKREIS       1554834 non-null  int64  
 3   UGEMEINDE    1554834 non-null  int64  
 4   UJAHR        1554834 non-null  int64  
 5   UMONAT       1554834 non-null  int64  
 6   USTUNDE      1554834 non-null  int64  
 7   UWOCHENTAG   1554834 non-null  int64  
 8   UKATEGORIE   1554834 non-null  int64  
 9   UART         1554834 non-null  int64  
 10  UTYP1        1554834 non-null  int64  
 11  ULICHTVERH   1554834 non-null  int64  
 12  STRZUSTAND   1554834 non-null  int64  
 13  IstRad       1554834 non-null  int64  
 14  IstPKW       1554834 non-null  int64  
 15  IstFuss      1554834 non-null  int64  
 16  IstKrad      1554834 non-null  int64  
 17  IstGkfz      1359605 non-null  float64
 18  IstSonst

In [7]:
merged_data.shape

(1554834, 23)

In [8]:
merged_data.describe()

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,UTYP1,ULICHTVERH,STRZUSTAND,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige
count,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.359605e+06,1.554834e+06
mean,7.431094e+00,2.788656e+00,4.191415e+01,6.952885e+01,2.019268e+03,6.766878e+00,1.331008e+01,4.102703e+00,2.793472e+00,3.901359e+00,3.819214e+00,4.294851e-01,2.764469e-01,2.878140e-01,7.866357e-01,8.647933e-02,1.412794e-01,4.938934e-02,1.111977e-01
std,3.588564e+00,2.349308e+00,2.588040e+01,1.116782e+02,1.918956e+00,3.214572e+00,4.776594e+00,1.868558e+00,4.288503e-01,2.608406e+00,2.140342e+00,7.880731e-01,4.945796e-01,4.527441e-01,4.096829e-01,2.810706e-01,3.483096e-01,2.166797e-01,3.143769e-01
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.016000e+03,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,1.000000e+00,1.500000e+01,0.000000e+00,2.018000e+03,4.000000e+00,1.000000e+01,3.000000e+00,3.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+00,3.000000e+00,3.900000e+01,2.000000e+01,2.019000e+03,7.000000e+00,1.400000e+01,4.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,9.000000e+00,4.000000e+00,6.300000e+01,1.130000e+02,2.021000e+03,9.000000e+00,1.700000e+01,6.000000e+00,3.000000e+00,5.000000e+00,6.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.600000e+01,9.000000e+00,9.100000e+01,7.180000e+02,2.022000e+03,1.200000e+01,2.300000e+01,7.000000e+00,3.000000e+00,9.000000e+00,7.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [9]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1554834 entries, 0 to 256491
Data columns (total 23 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   ULAND        1554834 non-null  int64  
 1   UREGBEZ      1554834 non-null  int64  
 2   UKREIS       1554834 non-null  int64  
 3   UGEMEINDE    1554834 non-null  int64  
 4   UJAHR        1554834 non-null  int64  
 5   UMONAT       1554834 non-null  int64  
 6   USTUNDE      1554834 non-null  int64  
 7   UWOCHENTAG   1554834 non-null  int64  
 8   UKATEGORIE   1554834 non-null  int64  
 9   UART         1554834 non-null  int64  
 10  UTYP1        1554834 non-null  int64  
 11  ULICHTVERH   1554834 non-null  int64  
 12  STRZUSTAND   1554834 non-null  int64  
 13  IstRad       1554834 non-null  int64  
 14  IstPKW       1554834 non-null  int64  
 15  IstFuss      1554834 non-null  int64  
 16  IstKrad      1554834 non-null  int64  
 17  IstGkfz      1359605 non-null  float64
 18  IstSonst

### Convert datatypes
Since the coordinates are saved as Strings we  have to convert them in a datatype we can later work with. 

In [10]:
merged_data['XGCSWGS84'] = merged_data['XGCSWGS84'].astype(str).str.replace(',', '.').astype(float)
merged_data['YGCSWGS84'] = merged_data['YGCSWGS84'].astype(str).str.replace(',', '.').astype(float)
merged_data['LINREFX'] = merged_data['LINREFX'].astype(str).str.replace(',', '.').astype(float)
merged_data['LINREFY'] = merged_data['LINREFY'].astype(str).str.replace(',', '.').astype(float)

merged_data.head(10)


,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,1,0,53,120,2016,1,9,5,2,8,...,0,1,0,0,0.0,0,606982.3940,5.954660e+06,10.621659,53.729615
1,1,0,57,10,2016,1,17,3,3,1,...,0,1,0,0,0.0,0,574882.5330,6.011441e+06,10.149176,54.245453
2,1,0,62,8,2016,1,0,5,3,9,...,0,1,0,0,0.0,0,599934.6875,5.964609e+06,10.518094,53.820403
3,1,0,3,0,2016,1,15,5,3,5,...,1,0,0,0,0.0,1,610709.3487,5.968284e+06,10.683021,53.851243
4,1,0,55,28,2016,1,14,1,3,8,...,0,1,0,0,0.0,0,605690.7904,6.009152e+06,10.620986,54.219459
5,1,0,61,44,2016,1,17,6,3,9,...,0,1,0,0,0.0,0,542547.3285,5.963922e+06,9.646338,53.822094
6,1,0,60,54,2016,1,17,3,3,6,...,1,0,1,0,0.0,0,558412.4075,5.969890e+06,9.888453,53.874190
7,1,0,55,29,2016,1,11,1,2,2,...,0,1,0,0,0.0,0,623911.7044,6.008211e+06,10.899896,54.206925
8,1,0,60,85,2016,1,16,2,2,4,...,0,1,0,0,0.0,0,579664.3081,5.965471e+06,10.210479,53.831656
9,1,0,54,126,2016,1,10,4,2,5,...,0,1,0,0,0.0,1,494320.5274,6.066810e+06,8.911766,54.748507


### Checking for Duplicates
Checking for duplicates by making a copy and doubling our dataset. Afterwards the doubles are being dropped. Are there no douplictates in the copied dataset, the original doesnt have duplicates either.

In [11]:
temp_df = merged_data.copy()
temp_df.drop_duplicates(inplace=True)
num_rows_merged_data = merged_data.shape[0]
num_rows_temp_df = temp_df.shape[0]

In [12]:

print("number of rows in merged_data: ", num_rows_merged_data)
print("number of rows in temp_df: ", num_rows_temp_df)    
print("difference in rows: ", num_rows_temp_df - num_rows_merged_data)


number of rows in merged_data:  1554834
number of rows in temp_df:  1554694
difference in rows:  -140


Since there is a difference of 140 we it is probible to have duplicates in our original dataset.

In [13]:
# show duplicates 
duplicates = merged_data[merged_data.duplicated(keep=False)]
duplicates.head(14)

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
52335,7,3,39,43,2016,3,13,4,3,1,...,0,1,0,0,0.0,0,453653.9008,5.523542e+06,8.355149,49.862495
52339,7,3,39,43,2016,3,13,4,3,1,...,0,1,0,0,0.0,0,453653.9008,5.523542e+06,8.355149,49.862495
54140,7,1,37,69,2016,6,5,5,3,2,...,0,1,0,0,0.0,0,376200.4225,5.583253e+06,7.258458,50.388285
54478,7,1,37,69,2016,6,5,5,3,2,...,0,1,0,0,0.0,0,376200.4225,5.583253e+06,7.258458,50.388285
58019,7,1,43,59,2016,9,14,5,3,2,...,0,1,0,0,0.0,0,409948.5422,5.595250e+06,7.730177,50.502286
58247,7,1,43,59,2016,9,14,5,3,2,...,0,1,0,0,0.0,0,409948.5422,5.595250e+06,7.730177,50.502286
58314,7,3,12,0,2016,10,21,7,3,9,...,0,1,0,1,0.0,0,406861.1510,5.478158e+06,7.715006,49.448939
58484,7,3,12,0,2016,10,21,7,3,9,...,0,1,0,1,0.0,0,406861.1510,5.478158e+06,7.715006,49.448939
60015,7,3,38,6,2016,12,19,1,3,2,...,0,1,0,0,0.0,0,452163.5267,5.473644e+06,8.340503,49.413595
60256,7,3,38,6,2016,12,19,1,3,2,...,0,1,0,0,0.0,0,452163.5267,5.473644e+06,8.340503,49.413595


The table shows that there are indeed duplicates in the dataset. Since the coordinates are exremly detailed, it is very unlilkely that the accdients happend the same time with the same particitpantson the exact same spot. After doublechecking the accidents on the map we decided to drop them.

In [14]:
#drop duplicates
merged_data.drop_duplicates(inplace=True)
num_rows_merged_data = merged_data.shape[0]

print("number of rows in merged_data: ", num_rows_merged_data)
print("number of rows in temp_df: ", num_rows_temp_df)
print("difference in rows: ", num_rows_temp_df - num_rows_merged_data)


number of rows in merged_data:  1554694
number of rows in temp_df:  1554694
difference in rows:  0


Since they have now the same amount of row, there are no further duplicates in the dataset.


## Renaming columns

In [15]:
merged_data.columns

Index(['ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR', 'UMONAT', 'USTUNDE',
       'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'STRZUSTAND',
       'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige',
       'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84'],
      dtype='object')

In [16]:
# Cols in lower case   
merged_data.columns =  [col.lower() for col in merged_data.columns]
merged_data.columns



Index(['uland', 'uregbez', 'ukreis', 'ugemeinde', 'ujahr', 'umonat', 'ustunde',
       'uwochentag', 'ukategorie', 'uart', 'utyp1', 'ulichtverh', 'strzustand',
       'istrad', 'istpkw', 'istfuss', 'istkrad', 'istgkfz', 'istsonstige',
       'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84'],
      dtype='object')

Since the U in the colnames stands for 'Unfall' and its obvious that we are having a dataset with data of accidents the U will be deleted to make things clearer. Furthermore the 1 in typ will be deleted and a _ will be added for the boolean categories.

In [17]:
merged_data.columns = merged_data.columns.str.lstrip('u')
merged_data.rename(columns=lambda x: x.replace('ist', 'ist_') if x.startswith('ist') else x, inplace=True)
merged_data.rename(columns={
    'typ1': 'typ',
    'lichtverh': 'licht' 
    }, inplace=True)

merged_data.columns


Index(['land', 'regbez', 'kreis', 'gemeinde', 'jahr', 'monat', 'stunde',
       'wochentag', 'kategorie', 'art', 'typ', 'licht', 'strzustand',
       'ist_rad', 'ist_pkw', 'ist_fuss', 'ist_krad', 'ist_gkfz',
       'ist_sonstige', 'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84'],
      dtype='object')

## Missing Values

In [18]:
# check for missing values
merged_data.isnull().sum()



land                 0
regbez               0
kreis                0
gemeinde             0
jahr                 0
monat                0
stunde               0
wochentag            0
kategorie            0
art                  0
typ                  0
licht                0
strzustand           0
ist_rad              0
ist_pkw              0
ist_fuss             0
ist_krad             0
ist_gkfz        195214
ist_sonstige         0
linrefx              0
linrefy              0
xgcswgs84            0
ygcswgs84            0
dtype: int64

As we can see the data set has almost no null values except for the binary variable 'ist_gkfz'. This will be investigated further and the data imputed where appropriate.

In [19]:
missing_data = merged_data[merged_data['ist_gkfz'].isnull()]
missing_data.sample(10)

,land,regbez,kreis,gemeinde,jahr,monat,stunde,wochentag,kategorie,art,...,ist_rad,ist_pkw,ist_fuss,ist_krad,ist_gkfz,ist_sonstige,linrefx,linrefy,xgcswgs84,ygcswgs84
111326,12,0,60,5,2017,4,14,5,3,5,...,1,1,0,0,NaN,0,812132.5277,5.838087e+06,13.610167,52.602931
64396,14,7,29,40,2017,9,13,6,3,2,...,0,1,0,1,NaN,0,736637.1311,5.677778e+06,12.387534,51.202315
97156,9,1,79,121,2017,11,6,6,3,5,...,0,1,0,0,NaN,0,666855.5242,5.339462e+06,11.244969,48.186459
84801,9,1,78,123,2017,7,17,4,3,2,...,0,1,0,0,NaN,0,690065.1351,5.360258e+06,11.566289,48.366848
79608,9,3,76,170,2017,5,15,2,3,0,...,0,1,0,0,NaN,0,727628.1409,5.454877e+06,12.125149,49.204371
77835,9,2,79,112,2017,4,11,2,2,7,...,0,0,0,0,NaN,1,758198.5681,5.395588e+06,12.506543,48.659922
111757,12,0,60,52,2017,5,10,6,3,0,...,0,1,0,0,NaN,0,824556.0418,5.865032e+06,13.819569,52.837039
131664,3,4,53,7,2017,7,18,5,3,5,...,1,1,0,0,NaN,0,423345.4690,5.874989e+06,7.857263,53.018946
143715,6,4,12,0,2017,9,10,3,3,5,...,0,1,0,1,NaN,0,477860.4567,5.556501e+06,8.690046,50.160314
87908,9,5,64,0,2017,7,19,7,3,2,...,0,1,0,0,NaN,0,651139.4957,5.467700e+06,11.080775,49.343277


In [20]:
missing_data_2017 = merged_data[merged_data['ist_gkfz'].isnull() & (merged_data['jahr'] == 2017)]
all_null_from_2017 = len(missing_data_2017) == len(missing_data)
not_null_from_2017 = merged_data[merged_data['ist_gkfz'].notnull() & merged_data['jahr'] == 2017].any().any()
print("All the null values for feature is_gkfz are from year 2017: ", all_null_from_2017)
print("There are notnull values for ist_gkfz in year 2017: ", not_null_from_2017)



All the null values for feature is_gkfz are from year 2017:  True
There are notnull values for ist_gkfz in year 2017:  False


Since all null values are only for the feature is_gkfz from 2017 and there are no notnull values for this feature in this year, it can be concluded that no data was collected for is_gkfz in this year.

In the following we will impute the missing values. We will check for patterns and correlation with other features. So we can train a logistic regression model, that imputes our missing values for the year 2017 

In [21]:
# Calculate the correlation matrix
correlation_matrix = merged_data.corr()

# Select the features with high correlation with is_gkfz
relevant_features = correlation_matrix['ist_gkfz'].abs().sort_values(ascending=False).index[1:]

# map correlation to each feature
correlation_to_ist_gkfz = {feature: correlation_matrix.loc['ist_gkfz', feature] for feature in relevant_features}


# Print the relevant features
print(correlation_to_ist_gkfz)


{'ist_rad': -0.10733862747940964, 'typ': 0.09456375198947359, 'stunde': -0.07848527764753568, 'ist_krad': -0.057456393171918876, 'ist_pkw': -0.057187447091856254, 'kategorie': -0.0519069042703419, 'ist_fuss': -0.046028361121884936, 'art': -0.0393140192823976, 'kreis': 0.02431316433109911, 'licht': -0.019836170951205146, 'gemeinde': 0.018754307961221503, 'wochentag': -0.013051362686227398, 'monat': -0.00970261731071049, 'jahr': -0.00842786096139455, 'linrefy': -0.0062591436233858395, 'ygcswgs84': -0.006115356146857279, 'land': 0.005696697403717505, 'regbez': 0.005381606881152162, 'ist_sonstige': -0.00532538672720455, 'linrefx': 0.003561234236690129, 'strzustand': 0.00337820718997781, 'xgcswgs84': 0.0032319654037303116}


In [29]:
imputation_data = merged_data.copy()

# delete data with missing values
imputation_data.dropna(inplace=True)

#defining the features and target
features = imputation_data.drop(['ist_gkfz', 'jahr', 'monat', 'land', 'regbez','xgcswgs84', 'ygcswgs84', 'linrefx', 'linrefy'],  axis=1)
target = imputation_data['ist_gkfz']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a scaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train = scaler.fit_transform(X_train)

# Transform the test data
X_test = scaler.transform(X_test)

# Create a logistic regression model
log_reg = LogisticRegression(max_iter=1000)

# Fit the model
log_reg.fit(X_train, y_train)

# Calculate accuracy score on test data
accuracy_score = accuracy_score(log_reg.predict(X_test), y_test)
print(accuracy_score)
precision_score = precision_score(log_reg.predict(X_test), y_test)
print(precision_score)




0.9556227381057463
0.11281975454679875


In [42]:
# Select the rows with missing values for ist_gkfz
missing_data = merged_data[merged_data['ist_gkfz'].isnull()]

# Extract the features for imputation
imputation_features = missing_data.drop(['ist_gkfz', 'jahr', 'monat', 'land', 'regbez', 'xgcswgs84', 'ygcswgs84', 'linrefx', 'linrefy'], axis=1)

# Scale the features using the previously created scaler
imputation_features_scaled = scaler.transform(imputation_features)

# Impute the missing values using the logistic regression model
imputed_values = log_reg.predict(imputation_features_scaled)

# Assign the imputed values to the missing rows in the merged_data dataframe
merged_data.loc[merged_data['ist_gkfz'].isnull(), 'ist_gkfz'] = imputed_values

# Check if there are any remaining missing values
merged_data.isnull().sum()


land            0
regbez          0
kreis           0
gemeinde        0
jahr            0
monat           0
stunde          0
wochentag       0
kategorie       0
art             0
typ             0
licht           0
strzustand      0
ist_rad         0
ist_pkw         0
ist_fuss        0
ist_krad        0
ist_gkfz        0
ist_sonstige    0
linrefx         0
linrefy         0
xgcswgs84       0
ygcswgs84       0
dtype: int64